<a href="https://colab.research.google.com/github/LarissaGao/Butterfly-Identification/blob/main/butterfly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/butterflies.zip

--2022-04-26 08:31:33--  https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/butterflies.zip
Resolving dphi-live.s3.eu-west-1.amazonaws.com (dphi-live.s3.eu-west-1.amazonaws.com)... 52.218.40.184
Connecting to dphi-live.s3.eu-west-1.amazonaws.com (dphi-live.s3.eu-west-1.amazonaws.com)|52.218.40.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239812069 (229M) [application/zip]
Saving to: ‘butterflies.zip’

butterflies.zip     100%[===================>] 228.70M  27.4MB/s    in 9.4s    

2022-04-26 08:31:43 (24.4 MB/s) - ‘butterflies.zip’ saved [239812069/239812069]



In [2]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [3]:
!7za x /content/butterflies.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 239812069 bytes (229 MiB)

Extracting archive: /content/butterflies.zip
 44% 4096 Open              --
Path = /content/butterflies.zip
Type = zip
Physical Size = 239812069

  0%      3% 306 - Butterfly Identification/test/Image_2711.jpg                                                         6% 640 - Butterfly Identification/test/Image_2522.jpg                                                         9% 916 - Butterfly Ide

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split

In [5]:
labels = pd.read_csv("/content/Butterfly Identification/Training_set.csv")
labels.head()

,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA
3,Image_4.jpg,MONARCH
4,Image_5.jpg,GREEN CELLED CATTLEHEART


In [6]:
labels.tail()

,filename,label
6494,Image_6495.jpg,MANGROVE SKIPPER
6495,Image_6496.jpg,MOURNING CLOAK
6496,Image_6497.jpg,APPOLLO
6497,Image_6498.jpg,ELBOWED PIERROT
6498,Image_6499.jpg,ATALA


In [7]:
file_paths = [[fname, '/content/Butterfly Identification/train/' + fname] for fname in labels['filename']]

In [8]:
if len(labels) == len(file_paths): 
  print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths)) 
else: 
  print('Number of labels does not match the number of filenames')

Number of labels i.e.  6499 matches the number of filenames i.e.  6499


In [9]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths']) 
images.head()

,filename,filepaths
0,Image_1.jpg,/content/Butterfly Identification/train/Image_...
1,Image_2.jpg,/content/Butterfly Identification/train/Image_...
2,Image_3.jpg,/content/Butterfly Identification/train/Image_...
3,Image_4.jpg,/content/Butterfly Identification/train/Image_...
4,Image_5.jpg,/content/Butterfly Identification/train/Image_...


In [10]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename') 
train_data.head()

,filename,filepaths,label
0,Image_1.jpg,/content/Butterfly Identification/train/Image_...,SOUTHERN DOGFACE
1,Image_2.jpg,/content/Butterfly Identification/train/Image_...,ADONIS
2,Image_3.jpg,/content/Butterfly Identification/train/Image_...,BROWN SIPROETA
3,Image_4.jpg,/content/Butterfly Identification/train/Image_...,MONARCH
4,Image_5.jpg,/content/Butterfly Identification/train/Image_...,GREEN CELLED CATTLEHEART


In [11]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
train_data['label'] = le.fit_transform(train_data['label'])

In [12]:
data = [] # initialize an empty numpy array 
image_size = 100 # image size taken is 100 here. one can take other size too 
for i in range(len(train_data)):  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) 
  # converting the image to gray scale  
  new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array 
  data.append([new_img_array, train_data['label'][i]])

In [13]:
4# image pixels of a image 
data[5]

[array([[ 65,  67,  69, ...,  85,  87,  88],
        [ 65,  66,  67, ...,  88,  87,  88],
        [ 60,  61,  65, ...,  88,  88,  90],
        ...,
        [111, 110, 111, ..., 114, 114, 117],
        [108, 110, 112, ..., 112, 113, 118],
        [109, 108, 111, ..., 111, 113, 121]], dtype=uint8), 14]

In [14]:
np.random.shuffle(data)

In [15]:
x = [] 
y = [] 
for image in data: 
 x.append(image[0]) 
 y.append(image[1])  
 
# converting x & y to numpy array as they are list 
x = np.array(x) 
y = np.array(y)

In [16]:
np.unique(y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74]),
 array([ 88,  75,  74,  85,  90, 100,  97,  83,  81,  85,  75,  86,  99,
         90,  83,  95,  85,  93,  87,  92,  87,  90,  94,  97,  72,  94,
         93,  92,  95,  82,  73,  78,  94,  88,  86,  81,  95,  81,  81,
         73,  87,  86,  76,  96,  90, 131,  87,  96,  76,  78,  90,  84,
         86,  84,  85,  79,  92,  77,  82,  96,  89,  86,  97,  83, 107,
         90,  87,  87,  83,  76,  84,  81,  71,  75,  76]))

In [17]:
x = x.reshape(-1, 100, 100, 1)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [19]:
cnn = tf.keras.models.Sequential([ 
                                  tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)), 
                                  tf.keras.layers.MaxPooling2D((2, 2)),  
                                  
                                  tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'), 
                                  tf.keras.layers.MaxPooling2D((2, 2)),  
                                  
                                  # tf.keras.layers.Flatten(input_shape=(100, 100, 1)), 
                                  tf.keras.layers.Flatten(), 
                                  tf.keras.layers.Dense(64, activation='relu'), 
                                  tf.keras.layers.Dense(75, activation='softmax')
                                   ])

In [20]:
cnn.compile(optimizer='adam', 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])

In [21]:
cnn.fit(X_train, y_train, epochs=2, batch_size=1)

Epoch 1/2
4549/4549 [==============================] - 123s 27ms/step - loss: 4.7084 - accuracy: 0.0207
Epoch 2/2
4549/4549 [==============================] - 124s 27ms/step - loss: 4.1066 - accuracy: 0.0607


In [22]:
cnn.evaluate(X_val, y_val)

61/61 [==============================] - 6s 101ms/step - loss: 4.0522 - accuracy: 0.0733


[4.052230358123779, 0.07333333045244217]

In [23]:
test_image_order = pd.read_csv("/content/Butterfly Identification/Testing_set.csv") 
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [24]:
file_paths = [[fname, '/content/Butterfly Identification/test/' + fname] for fname in test_image_order['filename']]

In [25]:
if len(test_image_order) == len(file_paths): 
  print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths)) 
else:
  print('Number of image names does not match the number of filepaths')

Number of image names i.e.  2786 matches the number of file paths i.e.  2786


In [26]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths']) 
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/Butterfly Identification/test/Image_1...
1,Image_2.jpg,/content/Butterfly Identification/test/Image_2...
2,Image_3.jpg,/content/Butterfly Identification/test/Image_3...
3,Image_4.jpg,/content/Butterfly Identification/test/Image_4...
4,Image_5.jpg,/content/Butterfly Identification/test/Image_5...


In [27]:
#test_data = pd.merge(test_images, labels, how = 'inner', on = 'filename') 
#test_data.head()

In [28]:
#from sklearn.preprocessing import LabelEncoder 
#le = LabelEncoder() 
#test_data['label'] = le.fit_transform(test_data['label'])

In [29]:
test_pixel_data = [] 
image_size = 100 

for i in range(len(test_images)): 

  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE) 
  new_img_array = cv2.resize(img_array, (image_size, image_size)) 
  test_pixel_data.append(new_img_array)

In [30]:
test_pixel_data = np.array(test_pixel_data)

In [31]:
test_pixel_data = test_pixel_data.reshape(-1, 100, 100, 1)

In [32]:
pred = cnn.predict(test_pixel_data)

In [33]:
pred[0]

array([0.00029335, 0.00130917, 0.00565439, 0.01387046, 0.00058701,
       0.01311231, 0.03274595, 0.01146584, 0.00255127, 0.001785  ,
       0.01269625, 0.01222533, 0.02276258, 0.00441425, 0.01272677,
       0.00091248, 0.0353495 , 0.00143538, 0.00251844, 0.00847234,
       0.01023741, 0.00819696, 0.00095002, 0.00455872, 0.0426694 ,
       0.01222552, 0.01848677, 0.00035345, 0.00084735, 0.01888935,
       0.00952807, 0.01501557, 0.015078  , 0.03676201, 0.00037848,
       0.02050168, 0.02387292, 0.00381534, 0.00094674, 0.00600417,
       0.02010973, 0.00345488, 0.00991394, 0.01991865, 0.00419745,
       0.03208803, 0.00855443, 0.00126045, 0.0118682 , 0.02053916,
       0.01929752, 0.0148325 , 0.00202785, 0.03440464, 0.0363714 ,
       0.00072086, 0.00074346, 0.01662507, 0.02297674, 0.0181795 ,
       0.03201447, 0.01947004, 0.00495792, 0.02037766, 0.00616792,
       0.02076526, 0.01849414, 0.01811792, 0.00606368, 0.02267966,
       0.02465001, 0.01099514, 0.00179005, 0.0241232 , 0.02504

In [34]:
prediction = [] 
for value in pred: 
  prediction.append(np.argmax(value))

In [35]:
predictions = le.inverse_transform(prediction)

In [36]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data 
res.to_csv("submission.csv", index = False)  

# To download the csv file locally 
from google.colab import files  
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>